<a href="https://colab.research.google.com/github/bskang8/CVPR2023_Project/blob/main/evi_gov_notice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install sentence-transformers
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=a5d223e44642cf8e169f84789b4bdb7d3fb81c21a8

In [2]:
!pip install xlrd
!pip install openpyxl
!pip install pandas

In [3]:
import os
import pandas as pd

from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

In [ ]:
file_path = '/content/drive/MyDrive/test_folder/dataset/서술형변환_chatGPT_줄별변환'
file_names = os.listdir(file_path)
chunk_documents = []

cut_len = 385
idx = 1
for file in file_names:
    whole_path = os.path.join(file_path, file)
    with open(whole_path, 'r', encoding='UTF8') as f:
        text = f.read()

    time = 1
    while time*cut_len <= len(text):
        b_time = time - 1
        chunk_text = text[b_time*cut_len:time*cut_len]
        chunk_documents.append(Document(page_content=chunk_text, metadata={'file_name':file, 'file_num': idx}))
        time += 1
    if (time-1)*cut_len <= len(text):
        chunk_text = text[(time-1)*cut_len:len(text)]
        chunk_documents.append(Document(page_content=chunk_text, metadata={'file_name':file, 'file_num': idx}))

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')

In [ ]:
vectordb = Chroma.from_documents(documents=chunk_documents, embedding=embeddings)

In [ ]:
doc_retriever = vectordb.as_retriever(search_kwargs={"k":5})

In [ ]:
def const_ref_doc(ref_docs):
  cont_ref = ''
  idx = 1
  for doc in ref_docs:
    cont_ref += "===== "
    cont_ref += ' Reference: ' + str(idx)
    cont_ref += " =====\n"
    cont_ref += doc.page_content
    idx += 1
    cont_ref += '\n'
    cont_ref += 'Meta Data : ' + doc.metadata['file_name']
    cont_ref += '\n'
  return cont_ref

In [4]:
file_name = '/content/drive/MyDrive/test_folder/dataset/qna/환경부 질의응답.xlsx'
df = pd.read_excel(file_name)
qs_list = df['질문'].tolist()

In [ ]:
s_path = '/content/drive/MyDrive/test_folder/dataset/contents_ver2'
os.makedirs(s_path, exist_ok=True)

for qs in qs_list:
  ref_docs = doc_retriever.get_relevant_documents(qs)
  ref = const_ref_doc(ref_docs)

  content = '#### Instruction: \n'
  content += '아래는 질문과 Reference 내용이다. Reference 내용을 기반으로 질문의 대답을 작성해 주시오.\n\n'
  content += '#### 질문: \n'
  content += qs
  content += '\n'
  content += '#### Reference: \n'
  content += ref
  content += '\n\n'
  content += '#### 대답: \n'

  qs_sep = qs.split('/')
  if len(qs_sep) > 1:
    qs_re = ''
    for s_q in qs_sep:
      qs_re += s_q
      qs_re += '_'
    qs_re[:len(qs_re)-1]
  else:
    qs_re = qs_sep[0]

  s_file_path = os.path.join(s_path, qs_re+'.txt')
  with open(s_file_path, 'w', encoding='UTF8') as f:
    f.write(content)

In [5]:
qs_list

['2023-581번 공고의 용역명은 무엇인가요?',
 '전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역의 용역기간은 얼마나 되나요?',
 '21,000,000원 이 있을때 전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역의 사업예산으로 충분한가요?',
 '전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역은 문서 개시후 몇일뒤 개찰 되었나요?',
 '전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역은 마지막 제출 후 몇 시간 후에 개찰되나요?',
 '환경부공고 제2023-537호의 용역명은 무엇인가요?',
 '환경부공고 제2023-537호의 용역기간은 얼마나 되나요?',
 '환경부공고 제2023-537호의 전자입찰서(가격)의 접수개시일시는 언제인가요?',
 '환경부공고 제2023-537호는 전자입찰서(가격)는 접수개시 후 몇일 뒤 마감되나요?',
 '환경부공고 제2023-537호의 개찰일시는 언제인가요?',
 '환경부공고 제2023-579호의 용역명은 무엇인가요?',
 '200,000,000원은 환경부공고 제2023-579호의 예산보다 큽니까?',
 '환경부공고 제2023-579호의 평가기관은 어디인가요?',
 '환경부공고 제2023-579호의 담당자 연락처는 무엇입니까?',
 '제안서 제출 후 어디에서 제안서 제출 유의사항을 확인할 수 있나요?',
 '환경부공고 제2023-536호의 용역기간은 얼마나 되나요?',
 '환경부공고 제2023-536호는 전자입찰서(가격)는 접수개시 후 몇일 뒤 마감되나요?',
 '환경부공고 제2023-536호의 개찰일시는 언제인가요?',
 '환경부공고 제2023-536호의 과업내용 및 제안서 관련 문의처는 어디인가요?',
 '환경부공고 제2023-536호의 입찰 및 계약사항 관련 문의처는 어디인가요?',
 '제5차 국가환경종합계획 정비를 위한 기초조사 연구의 공고 번호는 무엇입니까',
 '환경부공고 제 2023-54

In [27]:
file_path = '/content/drive/MyDrive/test_folder/dataset/contents'
file_names = os.listdir(file_path)

for file in file_names:
  whole_path = os.path.join(file_path, file)
  with open(whole_path, 'r', encoding='UTF8') as f:
    content = f.read()
    ext_qs = content[content.find("#### 질문")+10:content.find("#### Reference")-1]
    position = qs_list.index(ext_qs)
    print(position)

0
1
2
3
4
5
6
8
7
9
11
10
12
13
14
15
16
17
18
20
19
22
21
23
24
25
26
27
28
39
29
40
41
42
30
31
43
32
33
44
34
45
35
36
46
47
37
48
38
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92


In [28]:
df

,공고번호,공고명,질문,답변
0,2023-581,전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역,2023-581번 공고의 용역명은 무엇인가요?,전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역입니다.
1,2023-581,전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역,전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역의 용...,586일입니다.
2,2023-581,전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역,"21,000,000원 이 있을때 전기차 사용후 배터리 자원순환 클러스터 조성사업 건...","아니요, 사업예산은 21,550,000원(부가가치세 포함)이므로 21,000,000..."
3,2023-581,전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역,전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역은 문...,문서 개시일인 2023.10.10.로부터 10일 뒤인 2023.10.20.(금)에 ...
4,2023-581,전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역,전기차 사용후 배터리 자원순환 클러스터 조성사업 건설폐기물(소각대상) 처리용역은 마...,접수 마감 후 1시간 뒤에 개찰됩니다.
...,...,...,...,...
88,2023-545,한국형 녹색분류체계 4개 환경목표 경제활동 도출 연구,한국형 녹색 분류 체계 4개 환경목표 경제활동 도출 연구 공고에서 예외규정이 적용되...,비영리 법인이 입찰 찹여하는 경우 예외규정 적용
89,2023-545,한국형 녹색분류체계 4개 환경목표 경제활동 도출 연구,한국형 녹색 분류 체계 4개 환경목표 경제활동 도출 연구 공고에 대한 입찰서는 어디...,제안서 제출방식은 온라인 제출입니다. 입찰에 참여하는 자는 e-발주시스템으로 제안서...
90,2023-545,한국형 녹색분류체계 4개 환경목표 경제활동 도출 연구,한국형 녹색 분류 체계 4개 환경목표 경제활동 도출 연구 과제를 수행하는 데 드는 ...,"사업예산: 100,000,000원(부가가치세 포함) 으로 수익 실현 불가능"
91,2023-545,한국형 녹색분류체계 4개 환경목표 경제활동 도출 연구,한국형 녹색 분류 체계 4개 환경목표 경제활동 도출 연구 공고에 가격 입찰서와 제안...,"가격 입차서와 제안서는 순서에 상관없이 제출 가능하지만, 제안서는 가급적 마감일 전..."
